In [1]:
from datamanager import DataManager
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn import LSTMCell
import pandas as pd
import random

/home/msps9341012/mygym/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
datamanager = DataManager("/home/msps9341012/AGnews")
train_data, dev_data, test_data = datamanager.getdata(4, 199)
wv = datamanager.get_wordvector("/home/msps9341012/AAAI18-code/glove.42B.300d.txt")

10020 words not find in wordvector
80721 words in total


In [3]:
test_word=np.vstack(list(map(lambda x: x['words'],test_data)))
test_lenth=np.hstack(list(map(lambda x: x['lenth'],test_data)))
test_solution=np.vstack(list(map(lambda x: x['solution'],test_data)))

dev_word=np.vstack(list(map(lambda x: x['words'],dev_data)))
dev_lenth=np.hstack(list(map(lambda x: x['lenth'],dev_data)))
dev_solution=np.vstack(list(map(lambda x: x['solution'],dev_data)))

In [111]:
batchsize=5
max_lenth=199

In [91]:
class Policy_net:
    def __init__(self):
        
        self.state = tf.placeholder(dtype=tf.float32,shape=[None,600], name='state')
        self.input_x=tf.placeholder(dtype=tf.float32,shape=[None,300], name='input')
        
        self.actions = tf.placeholder(tf.int32, [None, ], name="actions_num")
        self.rewards=tf.placeholder(dtype=tf.float32,shape=[None], name='input')
        
        self.global_step = tf.Variable(0, trainable=False, name="ActorStep")
        self.learning_rate = tf.train.exponential_decay(0.0005, self.global_step, 10000, 0.95, staircase=True)
        
        with tf.variable_scope('policy_net'):
            layer_1 = tf.layers.dense(inputs=self.state, units=1, activation=tf.identity)
            layer_2 = tf.layers.dense(inputs=self.input_x, units=1, activation=tf.identity)
            self.scaled_out=tf.sigmoid(layer_1+layer_2)
            s_out = tf.clip_by_value(self.scaled_out, 1e-5, 1 - 1e-5)
            self.scaled_out = tf.concat([1-s_out,s_out],axis=1) # Policy act_prob

            self.act_stochastic = tf.multinomial(tf.log(self.scaled_out), num_samples=1) #[batch, n_class]
            self.act_stochastic = tf.reshape(self.act_stochastic, shape=[-1])

            self.act_deterministic = tf.argmax(self.scaled_out, axis=1)

            self.scope = tf.get_variable_scope().name
        #loss
        neg_log_prob = tf.reduce_sum(-tf.log(self.scaled_out)*tf.one_hot(self.actions, 2), axis=1)
        self.loss = tf.reduce_mean(neg_log_prob * self.rewards)
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss,global_step=self.global_step)
            
    def act(self,state,x,stochastic=True):
        if stochastic:
            return tf.get_default_session().run(self.act_stochastic, feed_dict={self.state: state,self.input_x:x})
        else:
            return tf.get_default_session().run(self.act_deterministic, feed_dict={self.state: state,self.input_x:x})

    #def get_action_prob(self, obs):
        #return tf.get_default_session().run(self.act_probs, feed_dict={self.obs: obs})

    def train(self,state, x, actions, rewards):
        return tf.get_default_session().run([self.loss,self.train_op], feed_dict={self.state: state,
                                                                      self.input_x:x,
                                                                      self.actions: actions,
                                                                      self.rewards: rewards})    
    
    def get_variables(self):
        return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, self.scope)

    def get_trainable_variables(self):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.scope)

In [103]:
class LSTM_CriticNetwork():
    def __init__(self, wordvector,dropout):
        with tf.variable_scope('Critic'):
            self.dropout=0.5
            self.global_step = tf.Variable(0, trainable=False, name="LSTMStep")
            self.learning_rate = tf.train.exponential_decay(0.0005, self.global_step, 10000, 0.95, staircase=True)

            self.keep_prob = tf.placeholder(tf.float32, name="keepprob")
            self.seq = tf.placeholder(shape=[None, max_lenth], dtype=tf.int32, name="input_seq") #一條

            self.cell_state=tf.placeholder(tf.float32, shape = [None, 600], name="cell_state")
            self.cell_input=tf.placeholder(tf.int32, shape = [None, 1], name="cell_input")

            self.lenth = tf.placeholder(shape=[None], dtype=tf.int32, name="lenth")
            self.wordvector = tf.get_variable('wordvector', dtype=tf.float32, initializer=wordvector, trainable=True)

            self.ground_truth = tf.placeholder(tf.float32, [None,4], name="ground_truth")
            self.init = tf.random_uniform_initializer(-0.05, 0.05, dtype=tf.float32)

            self.seq_vec = tf.nn.embedding_lookup(self.wordvector, self.seq)

            self.vec = tf.nn.embedding_lookup(self.wordvector, self.cell_input) #for one step

            self.L2regular = 0.00001
        
            with tf.variable_scope('Rep') as scope:
                self.cell = LSTMCell(300, initializer=self.init, state_is_tuple=False)
                self.output, state=tf.nn.dynamic_rnn(self.cell,self.seq_vec,dtype=tf.float32,sequence_length=self.lenth)
                batch_range = tf.range(tf.shape(self.lenth)[0])
                self.output=tf.gather_nd(self.output, tf.stack([batch_range, self.lenth-1], axis=1))#取出最後一個
                scope.reuse_variables()
                loss_norm_r=tf.nn.l2_loss(tf.get_variable('rnn/lstm_cell/kernel'))*self.L2regular
                
            with tf.variable_scope('CNet') as scope:
                self.layer1 = tf.layers.dropout(self.output, self.keep_prob)   # drop out 50% of inputs
                self.target_out = tf.layers.dense(self.layer1, 4) #output: 4
                scope.reuse_variables()
                loss_norm_c=tf.nn.l2_loss(tf.get_variable('dense/kernel'))*self.L2regular
            
            #one step            
            self.output_next, self.h_next = self.cell.call(self.vec[:,0,:], self.cell_state)
            self.scope = tf.get_variable_scope().name
            
        self.reward=tf.reduce_max(tf.multiply(tf.nn.softmax(self.target_out),self.ground_truth),axis=1)

        self.loss_target = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.ground_truth, logits=self.target_out))
        
        self.c_net_loss=self.loss_target+loss_norm_c
        self.r_net_loss=self.loss_target+loss_norm_r
        self.total_loss=self.loss_target+loss_norm_c+loss_norm_r
        
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.total_loss,global_step=self.global_step)
        
        self.pre_train_r=tf.train.AdamOptimizer(self.learning_rate).minimize(self.r_net_loss,global_step=self.global_step,
                                                                             var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'Critic/Rep'))
        
        self.pre_train_c=tf.train.AdamOptimizer(self.learning_rate).minimize(self.c_net_loss,global_step=self.global_step,
                                                                             var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'Critic/CNet'))
        
    def get_next_state(self,x,init_h):
        return tf.get_default_session().run([self.output_next,self.h_next], feed_dict={self.cell_input: x,
                                                                                      self.cell_state:init_h})
    def get_reward(self,sequence,lenth,ground_truth):
        return tf.get_default_session().run(self.reward, feed_dict={self.seq: sequence,
                                                                    self.lenth: lenth,
                                                                    self.ground_truth: ground_truth,
                                                                    self.keep_prob: 1.0})
    
    def get_wordvect(self, sequence):
        return tf.get_default_session().run(self.seq_vec, feed_dict={self.seq: sequence})
    
   
    def train(self, sequence, lenth, ground_truth):
        return tf.get_default_session().run([self.total_loss, self.train_op],feed_dict={self.seq: sequence,
                                                                         self.lenth: lenth,
                                                                         self.ground_truth: ground_truth,
                                                                         self.keep_prob: self.dropout})
    def predict(self, sequence, lenth):
        return tf.get_default_session().run(self.target_out, feed_dict={self.seq: sequence,
                                                         self.lenth: lenth,
                                                         self.keep_prob: 1.0})
    
    def pretrain_c(self, sequence, lenth, ground_truth):
        return tf.get_default_session().run([self.c_net_loss, self.pre_train_c],feed_dict={self.seq: sequence,
                                                                            self.lenth: lenth,
                                                                            self.ground_truth: ground_truth,
                                                                            self.keep_prob: self.dropout})
    
    def pretrain_r(self, sequence, lenth, ground_truth):
        return tf.get_default_session().run([self.r_net_loss, self.pre_train_r],feed_dict={self.seq: sequence,
                                                                            self.lenth: lenth,
                                                                            self.ground_truth: ground_truth,
                                                                            self.keep_prob: self.dropout})
    def get_trainable_variables(self):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, self.scope)

In [93]:
def sampling_RL(inputs, stochastic=True):
    #print epsilon
    current_lower_state = np.zeros((1, 2*300), dtype=np.float32) #initial state
    actions = []
    states = []
    words=inputs['words']
    vec=critic.get_wordvect([inputs['words']])[0]
    lenth=inputs['lenth']
    
    #choose actions
    for pos in range(lenth):
        action = actor.act(current_lower_state, [vec[pos]],stochastic)
        action=np.asscalar(action)
        states.append([current_lower_state[0], vec[pos]]) #store state

        actions.append(action)
        if action == 1: # 1 for retain
            out_d, current_lower_state = critic.get_next_state([[words[pos]]],current_lower_state) #可以進到一下步

    Rinput = []
    for (i, a) in enumerate(actions):
        if a == 1:
            Rinput.append(words[i])
    Rlenth = len(Rinput)
    if Rlenth == 0:
        actions[lenth-2] = 1
        Rinput.append(words[lenth-2])
        Rlenth = 1
    Rinput += [0] * (max_lenth - Rlenth) #補零
    
    return actions, states, Rinput, Rlenth

In [112]:
tf.reset_default_graph()
actor=Policy_net()
critic=LSTM_CriticNetwork(wv,0.5)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/home/msps9341012/mygym/lib/python3.6/site-packages/tensorflow/python/client/session.py:1662: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [95]:
def test(test_data,test_word,test_lenth,test_solution,noRL):
    acc = 0
    if noRL:
        for b in range(len(test_word) // 100):
            out = critic.predict(test_word[b * 100: (b+1) * 100], test_lenth[b * 100: (b+1) * 100])
            acc=acc+np.sum(np.argmax(out,axis=1)==np.argmax(test_solution[b*100:(b+1) * 100],axis=1))
            
    else:        
        for data in test_data:         
            actions, states, Rinput, Rlenth = sampling_RL(data,False)
            out = critic.predict([Rinput], [Rlenth])
            if np.argmax(out) == np.argmax(data['solution']):
                acc += 1
    return float(acc) / len(test_data)


In [ ]:
random.shuffle(train_data)
for b in range(len(train_data) // batchsize): #一個batch
    datas = train_data[b * batchsize: (b+1) * batchsize]
    total_loss=0
    for inputs in datas: 
        #if net=='cnet':
        loss, _ = critic.pretrain_c([inputs['words']], [inputs['lenth']], [inputs['solution']])
        total_loss=total_loss+loss
        #if net=='rep':
            #loss, _ = critic.pretrain_r(word_batch, lenth_batch, solution_batch)
        if (b+1)%1000==0:    
            acc_test = test(test_data,test_word,test_lenth,test_solution, noRL= True)
            acc_dev = test(dev_data,dev_word,dev_lenth,dev_solution, noRL= True)
            print("batch ",b , "total loss ", loss, "----test: ", acc_test, "| dev: ", acc_dev)
    else:
        actionlist, statelist, inputlist, rewards = [], [], [],[]
        for inputs in datas: #batch 一條一條跑
            actions, states, Rinput, Rlenth = sampling_RL(inputs,True)
            statelist.append(np.array(list(map(lambda x: x[0], states)))) #current_lower_state
            inputlist.append(np.array(list(map(lambda x: x[1], states))))
            reward=np.asscalar(critic.get_reward([Rinput], [Rlenth],[inputs['solution']]))
            reward += 0.05*4*(1- float(Rlenth)/inputs['lenth'])
            rewards.append(reward)


In [ ]:
pretrain('cnet')

batch  499 total loss  1.927728 ----test:  0.8475 | dev:  0.8442
batch  999 total loss  0.40529075 ----test:  0.8640789473684211 | dev:  0.8621
batch  1499 total loss  0.16963314 ----test:  0.8832894736842105 | dev:  0.8812
batch  1999 total loss  0.02860642 ----test:  0.8869736842105264 | dev:  0.8857
batch  2499 total loss  0.04875701 ----test:  0.89 | dev:  0.8904
batch  2999 total loss  0.10487152 ----test:  0.8905263157894737 | dev:  0.893
batch  3499 total loss  0.81759316 ----test:  0.8869736842105264 | dev:  0.8886
batch  3999 total loss  0.04382562 ----test:  0.8803947368421052 | dev:  0.8816
batch  4499 total loss  1.3453445 ----test:  0.895 | dev:  0.8968
batch  4999 total loss  0.98137546 ----test:  0.8935526315789474 | dev:  0.8949
batch  5499 total loss  0.88240844 ----test:  0.9023684210526316 | dev:  0.9
batch  5999 total loss  0.18125913 ----test:  0.9040789473684211 | dev:  0.9038
batch  6499 total loss  0.016127413 ----test:  0.9048684210526315 | dev:  0.9081
batch  

In [113]:
def pretrain(net):
    random.shuffle(train_data)
    for b in range(len(train_data) // batchsize): #一個batch
        datas = train_data[b * batchsize: (b+1) * batchsize]
        if net =='cnet':
            word_batch=np.vstack(list(map(lambda x: x['words'],datas)))
            lenth_batch=np.hstack(list(map(lambda x: x['lenth'],datas)))
            solution_batch=np.vstack(list(map(lambda x: x['solution'],datas)))
            loss, _ = critic.train(word_batch, lenth_batch, solution_batch)
            if (b+1)%500==0:    
                acc_test = test(test_data,test_word,test_lenth,test_solution, noRL= True)
                acc_dev = test(dev_data,dev_word,dev_lenth,dev_solution, noRL= True)
                print("batch ",b , "total loss ", loss, "----test: ", acc_test, "| dev: ", acc_dev)
        else:
            actionlist, statelist, inputlist, rewards = [], [], [],[]
            for inputs in datas: #batch 一條一條跑
                actions, states, Rinput, Rlenth = sampling_RL(inputs,True)
                statelist.append(np.array(list(map(lambda x: x[0], states)))) #current_lower_state
                inputlist.append(np.array(list(map(lambda x: x[1], states))))
                reward=np.asscalar(critic.get_reward([Rinput], [Rlenth],[inputs['solution']]))
                reward += 0.05*4*(1- float(Rlenth)/inputs['lenth'])
                rewards.append(reward)
            
            loss, _=actor.train(np.vstack(statelist),np.vstack(inputlist),np.hstack(actionlist),np.hstack(rewards)) 
            if (b+1)%1000==0:    
                acc_test = test(test_data, noRL= False)
                acc_dev = test(dev_data, noRL= False)
                print("batch ",b , "total loss ", loss, "----test: ", acc_test, "| dev: ", acc_dev)

In [ ]:
#Main
print('---Main Train---')
random.shuffle(train_data)
for b in range(len(train_data) // batchsize): #一個batch
        datas = train_data[b * batchsize: (b+1) * batchsize]
        totloss = 0.
        actionlist, statelist, inputlist, rewards = [], [], [],[]
        Rinput_list=[]
        Rlenth_list=[]
        solution_list=[]
        for inputs in datas:
            actions, states, Rinput, Rlenth = sampling_RL(inputs,True)
            Rinput_list.append(Rinput)
            Rlenth_list.append(Rlenth)
            solution_list.append(solution_list)
            actionlist.append(actions)
            statelist.append(states)
            reward=np.asscalar(critic.get_reward([Rinput], [Rlenth],[inputs['solution']]))
            reward += 0.05*4*(1- float(Rlenth)/inputs['lenth'])
            rewards.append(reward)
            
        critic_loss, _ = critic.train(np.vstack(Rinput_list), np.hstack(Rlenth_list), np.vstack(solution_list))
        
        actor_loss, _=actor.train(np.vstack(statelist),np.vstack(inputlist),np.hstack(actionlist),np.hstack(rewards))
                             
        if (b+1)%1000==0:
            acc_test = test(test_data, noRL= False)
            acc_dev = test(dev_data, noRL= False)
            print("batch ",b , "total loss ", critic_loss+actor_loss, "----test: ", acc_test, "| dev: ", acc_dev)

In [ ]:


def train(sess, actor, critic, train_data, batchsize, samplecnt=5, LSTM_trainable=True, RL_trainable=True):
    print "training : total ", len(train_data), "nodes."
    random.shuffle(train_data)
    for b in range(len(train_data) / batchsize):
        datas = train_data[b * batchsize: (b+1) * batchsize]
        totloss = 0.
        critic.assign_active_network()
        actor.assign_active_network()
        for j in range(batchsize):
            #prepare
            data = datas[j]
            inputs, solution, lenth = data['words'], data['solution'], data['lenth']
            #train the predict network
            if RL_trainable:
                actionlist, statelist, losslist = [], [], []
                aveloss = 0.
                for i in range(samplecnt):
                    actions, states, Rinput, Rlenth = sampling_RL(sess, actor, inputs, critic.wordvector_find([inputs]), lenth, args.epsilon, Random=True)
                    actionlist.append(actions)
                    statelist.append(states)
                    out, loss = critic.getloss([Rinput], [Rlenth], [solution])
                    loss += (float(Rlenth) / lenth) **2 *0.15
                    aveloss += loss
                    losslist.append(loss)
                
                aveloss /= samplecnt
                totloss += aveloss
                grad = None
                if LSTM_trainable:
                    out, loss, _ = critic.train([Rinput], [Rlenth], [solution])
                for i in range(samplecnt):
                    for pos in range(len(actionlist[i])):
                        rr = [0., 0.]
                        rr[actionlist[i][pos]] = (losslist[i] - aveloss) * args.alpha
                        g = actor.get_gradient(statelist[i][pos][0], statelist[i][pos][1], rr)
                        if grad == None:
                            grad = g
                        else:
                            grad[0] += g[0]
                            grad[1] += g[1]
                            grad[2] += g[2]
                actor.train(grad)
            else: #Pretrain Critic 用
                out, loss, _ = critic.train([inputs], [lenth], [solution])
                totloss += loss
        if RL_trainable:
            actor.update_target_network()
            if LSTM_trainable:
                critic.update_target_network()
        else:
            critic.assign_target_network()
        if (b + 1) % 500 == 0:
            acc_test = test(sess, actor, critic, test_data, noRL= not RL_trainable)
            acc_dev = test(sess, actor, critic, dev_data, noRL= not RL_trainable)
            print("batch ",b , "total loss ", totloss, "----test: ", acc_test, "| dev: ", acc_dev)


def test(sess, actor, critic, test_data, noRL=False):
    acc = 0
    for i in range(len(test_data)):
        #prepare
        data = test_data[i]
        inputs, solution, lenth = data['words'], data['solution'], data['lenth']
        
        #predict
        if noRL:
            out = critic.predict_target([inputs], [lenth])
        else:
            actions, states, Rinput, Rlenth = sampling_RL(sess, actor, inputs, critic.wordvector_find([inputs]), lenth, Random=False)
            out = critic.predict_target([Rinput], [Rlenth])
        if np.argmax(out) == np.argmax(solution):
            acc += 1
    return float(acc) / len(test_data)


### Main ###
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config = config) as sess:
    #model
    critic = LSTM_CriticNetwork(sess, args.dim, args.optimizer, args.lr, args.tau, args.grained, args.maxlenth, args.dropout, word_vector) 
    actor = ActorNetwork(sess, args.dim, args.optimizer, args.lr, args.tau)
    #print variables
    for item in tf.trainable_variables():
        print (item.name, item.get_shape())
    
    saver = tf.train.Saver()
    
    #LSTM pretrain
    if args.RLpretrain != '':
        pass
    elif args.LSTMpretrain == '':
        sess.run(tf.global_variables_initializer())
        for i in range(0, 2):
            train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt, RL_trainable=False)
            critic.assign_target_network()
            acc_test = test(sess, actor, critic, test_data, True)
            acc_dev = test(sess, actor, critic, dev_data, True)
            print ("LSTM_only ",i, "----test: ", acc_test, "| dev: ", acc_dev)
            saver.save(sess, "checkpoints/"+args.name+"_base", global_step=i)
        print ("LSTM pretrain OK")
    else:
        print ("Load LSTM from ", args.LSTMpretrain)
        saver.restore(sess, args.LSTMpretrain)
    
    print ("epsilon", args.epsilon)

    if args.RLpretrain == '':
        for i in range(0, 5):
            train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt, LSTM_trainable=False)
            acc_test = test(sess, actor, critic, test_data)
            acc_dev = test(sess, actor, critic, dev_data)
            print("RL pretrain ", i, "----test: ", acc_test, "| dev: ", acc_dev)
            saver.save(sess, "checkpoints/"+args.name+"_RLpre", global_step=i)
        print ("RL pretrain OK")
    else:
        print ("Load RL from", args.RLpretrain)
        saver.restore(sess, args.RLpretrain)

    for e in range(args.epoch):
        train(sess, actor, critic, train_data, args.batchsize, args.sample_cnt)
        acc_test = test(sess, actor, critic, test_data)
        acc_dev = test(sess, actor, critic, dev_data)
        print ("epoch ", e, "----test: ", acc_test, "| dev: ", acc_dev)
        saver.save(sess, "checkpoints/"+args.name, global_step=e)


